In [1]:
import FABRIC

params = FABRIC.loadArgs('./params/params.yaml')
FABRIC.genFabDf(params)

[+] FABRIC: genFabDf marked as deactive, modify param.yaml for activation.


In [2]:
import pandas as pd
df = pd.read_pickle('/esplabdata/FABRIC/prc/UNIVERSAL_FABRIC.pkl.gz', compression='gzip')

In [3]:
df.head(5)

,subId,expType,expStage,time (s),expTrial,expName,stabilityStatus,eyeStatus,Fx,Fy,Fz,Mx,My,Mz
0,1008,MP,POST,0.001,1,Balance,Stable,Closed,1.442505,-0.152435,756.687511,-13.425294,5.799073,-4.725342
1,1008,MP,POST,0.002,1,Balance,Stable,Closed,1.401611,0.162674,755.218761,-13.843751,5.957276,-4.687744
2,1008,MP,POST,0.003,1,Balance,Stable,Closed,1.341858,-0.508728,756.500011,-13.707520,5.773194,-4.687988
3,1008,MP,POST,0.004,1,Balance,Stable,Closed,1.299622,0.070721,757.375011,-13.692872,5.678223,-4.788330
4,1008,MP,POST,0.005,1,Balance,Stable,Closed,1.526489,0.116745,755.437511,-13.713868,5.732666,-4.697022


In [6]:
df

,subId,expType,expStage,time (s),expTrial,expName,stabilityStatus,eyeStatus,Fx,Fy,Fz,Mx,My,Mz,New
0,1008,MP,POST,0.001,1,Balance,Stable,Closed,1.442505,-0.152435,756.687511,-13.425294,5.799073,-4.725342,1.290070
1,1008,MP,POST,0.002,1,Balance,Stable,Closed,1.401611,0.162674,755.218761,-13.843751,5.957276,-4.687744,1.564285
2,1008,MP,POST,0.003,1,Balance,Stable,Closed,1.341858,-0.508728,756.500011,-13.707520,5.773194,-4.687988,0.833130
3,1008,MP,POST,0.004,1,Balance,Stable,Closed,1.299622,0.070721,757.375011,-13.692872,5.678223,-4.788330,1.370342
4,1008,MP,POST,0.005,1,Balance,Stable,Closed,1.526489,0.116745,755.437511,-13.713868,5.732666,-4.697022,1.643234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272275995,94,SM,SD3,29.996,2,Balance,Unstable,Open,11.571289,1.227844,898.031264,-3.679932,2.320923,-1.170349,12.799133
272275996,94,SM,SD3,29.997,2,Balance,Unstable,Open,11.125000,1.334717,899.125014,-4.096436,2.147583,-1.041565,12.459717
272275997,94,SM,SD3,29.998,2,Balance,Unstable,Open,10.755371,0.721161,898.406264,-3.702027,2.496460,-1.096558,11.476532
272275998,94,SM,SD3,29.999,2,Balance,Unstable,Open,11.118653,0.970551,897.687514,-3.628296,2.304810,-1.133423,12.089203
